# Control of mountain car using defferential semi-gradient Sarsa with tiling
# by Ehsan Kouchaki


In [ ]:
import gym
import numpy as np
env = gym.make("MountainCar-v0")
import time

# Tiling Code from Sutton: http://incompleteideas.net/tiles/tiles3.py-remove

In [ ]:
basehash = hash

class IHT:
    "Structure to handle collisions"
    def __init__(self, sizeval):
        self.size = sizeval                        
        self.overfullCount = 0
        self.dictionary = {}

    def __str__(self):
        "Prepares a string for printing whenever this object is printed"
        return "Collision table:" + \
               " size:" + str(self.size) + \
               " overfullCount:" + str(self.overfullCount) + \
               " dictionary:" + str(len(self.dictionary)) + " items"

    def count (self):
        return len(self.dictionary)
    
    def fullp (self):
        return len(self.dictionary) >= self.size
    
    def getindex (self, obj, readonly=False):
        d = self.dictionary
        if obj in d: return d[obj]
        elif readonly: return None
        size = self.size
        count = self.count()
        if count >= size:
            if self.overfullCount==0: print('IHT full, starting to allow collisions')
            self.overfullCount += 1
            return basehash(obj) % self.size
        else:
            d[obj] = count
            return count

def hashcoords(coordinates, m, readonly=False):
    if type(m)==IHT: return m.getindex(tuple(coordinates), readonly)
    if type(m)==int: return basehash(tuple(coordinates)) % m
    if m==None: return coordinates

from math import floor, log
from itertools import zip_longest

def tiles (ihtORsize, numtilings, floats, ints=[], readonly=False):
    """returns num-tilings tile indices corresponding to the floats and ints"""
    qfloats = [floor(f*numtilings) for f in floats]
    Tiles = []
    for tiling in range(numtilings):
        tilingX2 = tiling*2
        coords = [tiling]
        b = tiling
        for q in qfloats:
            coords.append( (q + b) // numtilings )
            b += tilingX2
        coords.extend(ints)
        Tiles.append(hashcoords(coords, ihtORsize, readonly))
    return Tiles

# Train

In [ ]:
d = 4096
W = np.zeros((d))
X = np.zeros((d))
TilingNum = 8            #number of tilings
alpha = 0.1/TilingNum    # step size
gamma = 1
q_hat = np.zeros((3))
iht=IHT(d)

stability = 0
for ep in range(500):
    state = env.reset()    
    for ac in range(3):
        feature_idx = tiles(iht, TilingNum, [TilingNum*state[0]/(0.5+1.2),
                                             TilingNum*state[1]/(0.07+0.07)], [ac])
        for i in range(len(feature_idx)):
            q_hat[ac] += W[feature_idx[i]]            
    action = np.argmax(q_hat)
    if stability > 20:
        print('\n The learning prosecc terminated in episode number: ', ep, 'step number: ', step)
        break
    step = 0        
    while True:
        step += 1
        state_prim, R, done, _ = env.step(action)        
        delta = R - q_hat[action]
        alpha_delta = alpha * delta
        
        Feature_idx = tiles(iht, TilingNum, [TilingNum*state[0]/(0.5+1.2),
                                             TilingNum*state[1]/(0.07+0.07)], [action])
        if done == True:       # if we are in terminal state
            X = np.zeros((d))
            for i in range(len(Feature_idx)):
                X[Feature_idx[i]] = 1            
            W += alpha_delta * X
            if step < 200:
                stability += 1
            else:
                stability = 0
            print(env.step(action), 'step = ', step)
            
            break
            
        q_hat = np.zeros((3))
        for ac in range(3):
            feature_idx = tiles(iht, TilingNum, [TilingNum*state_prim[0]/(0.5+1.2),
                                                 TilingNum*state_prim[1]/(0.07+0.07)], [ac])
            for i in range(len(feature_idx)):
                q_hat[ac] += W[feature_idx[i]]            
        action = np.argmax(q_hat)
        delta += gamma*np.max(q_hat)
        alpha_delta = alpha * delta
        for i in range(len(Feature_idx)):
            W[Feature_idx[i]] += alpha_delta
        state = state_prim


# Test

In [ ]:
for ep in range(20):
    state = env.reset()    
    step = 0
    while True:
        step += 1
        env.render()
        time. sleep(.01)
        state_prim, R, done, _ = env.step(action) 
        
        Feature_idx = tiles(iht, TilingNum, [TilingNum*state[0]/(0.5+1.2),
                                             TilingNum*state[1]/(0.07+0.07)], [action])
        if done == True:       # if we are in terminal state
            print('The state in episode: ', ep + 1, 'is: ', env.step(action), 'step = ', step)
            break
            
        q_hat = np.zeros((3))
        for ac in range(3):
            feature_idx = tiles(iht, TilingNum, [TilingNum*state_prim[0]/(0.5+1.2),
                                                 TilingNum*state_prim[1]/(0.07+0.07)], [ac])
            for i in range(len(feature_idx)):
                q_hat[ac] += W[feature_idx[i]]            
        action = np.argmax(q_hat)
        state = state_prim
env.close()